In [1]:
%set_env CUDA_DEVICE_ORDER=PCI_BUS_ID
%set_env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torchvision
import argparse
from torch.autograd import Variable
import torch.utils.data as data

#from data import v2, v1, AnnotationTransform, VOCDetection, detection_collate, VOCroot, VOC_CLASSES
from data import v2, v1, detection_collate
from data import MSCOCODetection, COCOAnnotationTransform, COMMON_CLASSES
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import numpy as np
import time
import sys
from logger import Logger

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

#hyperparameters

version = "v2"
basenet = "vgg16_reducedfc.pth"
jaccard_threshold=0.5
lr=8e-4

save_folder="./MSCOCO14weights/"

voc_root= "/new_data/gpu/utkrsh/coco/" # location of the image root directory

annFile = "/new_data/gpu/utkrsh/coco/annotations/instances_train2014.json"
train_img = "/new_data/gpu/utkrsh/coco/images/train2014/"

cuda=True
# resume = "./MSCOCO14weights/ssd300_COCO_8000.pth" # saved trained weights
resume = None
start_iter = 1

if cuda and torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

if not os.path.exists(save_folder):
    os.mkdir(save_folder)
    
cfg = (v1, v2)[version == 'v2']

#train_sets = [('2007', 'trainval'), ('2012', 'trainval')]
# train_sets = 'train'
ssd_dim = 300  # only support 300 now
means = (104, 117, 123)  # imagenet mean values
num_classes = len(COMMON_CLASSES) + 1
batch_size = 32
accum_batch_size = 32
iter_size = accum_batch_size / batch_size
max_iter = 50000
weight_decay = 0.0005
stepvalues = (4000, 80000, 120000, 140000)
lr_steps = { 4000 : 0.001,
           80000 : 0.0001,
           120000 : 0.00001
           }
gamma = 0.1
momentum = 0.9

In [3]:
ssd_net = build_ssd('train', 300, num_classes)
net = ssd_net


if resume:
    print("Resuming training, loading weights from {}...".format(resume))
    ssd_net.load_weights(resume)
else:
    vgg_weights = torch.load(save_folder + basenet)
    print('Loading base network...')
    ssd_net.vgg.load_state_dict(vgg_weights)

if cuda:
    net = net.cuda()
#     net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True

def xavier(param):
    init.xavier_uniform(param)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

if not resume:
    print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    ssd_net.extras.apply(weights_init)
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=momentum, weight_decay=weight_decay)
criterion = MultiBoxLoss(num_classes, 0.5, True, 0, True, 3, 0.5, False, cuda)

def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    #local_lr = lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr_steps[step]

Loading base network...
Initializing weights...


In [4]:
logger = Logger(".logs/onlyCOCO14_baseline_model/")

def to_np(x):
    return x.data.cpu().numpy()

In [5]:
def my_detection_collate(batch):
    """Custom collate fn for dealing with batches of images that have a different
    number of associated object annotations (bounding boxes).

    Arguments:
        batch: (tuple) A tuple of tensor images and lists of annotations

    Return:
        A tuple containing:
            1) (tensor) batch of images stacked on their 0 dim
            2) (list of tensors) annotations for a given image are stacked on 0 dim
    """
    targets_1 = []
    imgs = []
    for sample in batch:
        # each sample is the result of one query on the dataset object
        imgs.append(sample[0])
        targets_1.append(torch.FloatTensor(sample[1]))
    return torch.stack(imgs, 0), targets_1


In [6]:
log_tf = True

In [7]:
def train():
    net.train()
    # loss counters
    loc_loss = 0  # epoch
    conf_loss = 0
    epoch = 0
    print_interval = 5
    curr_skip = 0
    max_skip = 2
    save_weights = True
    st_values = lr_steps.keys()
    
    print('Loading Dataset...')
    
    dataset = MSCOCODetection(image_root=train_img, ann_root=annFile, 
                              transform=SSDAugmentation(ssd_dim, means), 
                              target_transform=COCOAnnotationTransform())
    #dataset = MSCOCODetection(image_root=train_img, ann_root=annFile)
    print("Dataset Loaded!")
    
    epoch_size = len(dataset) // batch_size
    print("Training SSD on", dataset.name)
    step_index = 0
    batch_iterator = None
    data_loader = data.DataLoader(dataset, batch_size, num_workers=4,
                             shuffle=True, collate_fn=my_detection_collate,
                             pin_memory=cuda)
    
    for iteration in range(start_iter, max_iter):
        if(not batch_iterator) or (iteration % epoch_size == 0):
            # create batch_iterator
            batch_iterator = iter(data_loader)
        if iteration in st_values:
            step_index += 1
            adjust_learning_rate(optimizer, gamma, iteration)
            # reset epoch loss counters
            epoch += 1
        
        images, targets = next(batch_iterator)
        if cuda:
            images = Variable(images.cuda())
            targets = [Variable(anno.cuda(), volatile=True) for anno in targets]
        else:
            images = Variable(images)
            targets = [Variable(anno, volatile=True) for anno in targets]
        
        # forward pass
        t0 = time.time()
        out = net(images)
        
        # backprop
        optimizer.zero_grad()
        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c
        loss.backward()
        optimizer.step()
        if log_tf:
            info = {
                'loc_loss' : loss_l.data[0],
                'conf_loss' : loss_c.data[0],
                'loss' : loss.data[0]
            }
            for k, val in info.items():
                logger.scalar_summary(k,val,iteration)
            for k, val in net.named_parameters():
                k = k.replace('.','/')
                logger.histo_summary(k, to_np(val), iteration)
                logger.histo_summary(k+'/grad', to_np(val.grad), iteration)
        t1 = time.time()
        if iteration % print_interval == 0:
            print("Timer: %.4f sec. " % (t1 - t0))
            print("iter: "+ repr(iteration) + "|| loss_loc: %.4f || loss_conf: %.4f || loss: %.4f || " 
                  % (loss_l.data[0], loss_c.data[0], loss.data[0]), end=' ')
        if iteration % 5000 == 0 and save_weights:
            try:
                print(" Saving state, iter: ", iteration)
                torch.save(ssd_net.state_dict(), "./MSCOCO14weights/ssd300_COCO_" +
                          repr(iteration) + ".pth")
            except:
                print("cannot save the file")
    torch.save(ssd_net.state_dict(), "./MSCOCO14weights/ssd300_COCO_final_model.pth")

In [8]:
train()

Loading Dataset...
loading annotations into memory...
Done (t=9.70s)
creating index...
index created!
Dataset Loaded!
Training SSD on COCO2014


/users/gpu/utkrsh/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/autograd/_functions/tensor.py:450: UserWarning: mask is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return tensor.masked_fill_(mask, value)


Timer: 1.6882 sec. 
iter: 5|| loss_loc: 3.6722 || loss_conf: 14.7035 || loss: 18.3757 ||  Timer: 1.6504 sec. 
iter: 10|| loss_loc: 3.1133 || loss_conf: 13.2061 || loss: 16.3195 ||  Timer: 1.6303 sec. 
iter: 15|| loss_loc: 3.0033 || loss_conf: 12.3313 || loss: 15.3345 ||  Timer: 1.7098 sec. 
iter: 20|| loss_loc: 3.5523 || loss_conf: 12.8119 || loss: 16.3642 ||  Timer: 1.6429 sec. 
iter: 25|| loss_loc: 3.5475 || loss_conf: 12.6853 || loss: 16.2328 ||  Timer: 1.6204 sec. 
iter: 30|| loss_loc: 4.2634 || loss_conf: 12.5221 || loss: 16.7855 ||  Timer: 1.6896 sec. 
iter: 35|| loss_loc: 3.3403 || loss_conf: 12.5912 || loss: 15.9315 ||  Timer: 1.6920 sec. 
iter: 40|| loss_loc: 3.4605 || loss_conf: 12.3936 || loss: 15.8541 ||  Timer: 1.6551 sec. 
iter: 45|| loss_loc: 3.6786 || loss_conf: 9.9091 || loss: 13.5877 ||  Timer: 1.6812 sec. 
iter: 50|| loss_loc: 3.2842 || loss_conf: 9.4374 || loss: 12.7216 ||  Timer: 1.6979 sec. 
iter: 55|| loss_loc: 4.3822 || loss_conf: 11.4375 || loss: 15.8197 ||  Ti

Process Process-1:
KeyboardInterrupt
Process Process-3:
Process Process-2:
Process Process-4:
Traceback (most recent call last):
  File "/users/gpu/utkrsh/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/users/gpu/utkrsh/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/users/gpu/utkrsh/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/users/gpu/utkrsh/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 40, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/new_data/gpu/utkrsh/DomainAdaptation/data/coco2014.py", line 81, in __getitem__
    im, gt, _, _ = self.pull_item(index)
  File "/new_data/gpu/utkrsh/DomainAdaptation/data/coco2014.py",

KeyboardInterrupt: 

%env CUDA_LAUNCH_BLOCKING=1

d_iter = iter(data_loader)
d_img, d_targets = next(d_iter)
if cuda:
    d_img = Variable(d_img.cuda())
else:
    d_img = Variable(d_img)
d_out = net(d_img)
if cuda:
    d_targets = [Variable(anno.cuda(), volatile=True) for anno in d_targets]
else:
    d_targets = [Variable(anno, volatile=True) for anno in d_targets]

print(d_targets)

import traceback

try:
    d_loss1, d_loss2 = criterion(d_out, d_targets)
except:
    traceback.print_exc()